In [7]:
# Current state of the loans task 

connector = RDSDatabaseConnector('/Users/ptm/Desktop/customer loans/credentials.yaml')

# Extracting the required data from the database
table_name = 'loan_payments'  
data = connector.extract_data_from_table(table_name)

# Calculating the percentage of loans recovered against the investor funding
data['recovery_percentage'] = ((data['recoveries'] + data['collection_recovery_fee']) / data['funded_amount_inv']) * 100

# Calculating the total amount funded
total_funded_amount = data['funded_amount_inv'].sum()

# Calculating the total amount that would be recovered up to 6 months in the future
total_recovery_6_months = data[data['loan_status'] == 'Fully Paid']['total_rec_prncp'].sum()

# Calculating the percentage of the total amount recovered up to 6 months in the future
percentage_recovery_6_months = (total_recovery_6_months / total_funded_amount) * 100

# Creating a pie chart to visualise the results


labels = ['Recovered', 'Not Recovered']
sizes = [percentage_recovery_6_months, 100 - percentage_recovery_6_months]
colors = ['#ff9999', '#66b3ff']

plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.title('Percentage of Total Amount Recovered up to 6 Months')
plt.show()


NameError: name 'connect_data_filtered' is not defined

In [ ]:



# Calculating loss 

# Calculating the percentage of charged-off loans historically
charged_off_loans = connect_data_filtered[connect_data_filtered['loan_status'] == 'Charged Off']
percentage_charged_off = (len(charged_off_loans) / len(connect_data_filtered)) * 100

# Calculating the total amount paid towards charged-off loans
total_paid_charged_off = charged_off_loans['total_payment'].sum()

print(f"Percentage of Charged Off Loans: {percentage_charged_off:.2f}%")
print(f"Total Amount Paid Towards Charged Off Loans: ${total_paid_charged_off:.2f}")

In [ ]:
# Calculating projected loss 

# Calculating the remaining term
charged_off_loans['remaining_term'] = charged_off_loans['loan_amount'] - charged_off_loans['total_rec_prncp']

# Calculating the projected loss
charged_off_loans['remaining_loss'] = charged_off_loans['recoveries'] + charged_off_loans['collection_recovery_fee']

# Creating a visalisation of the projected loss over the remaining term
plt.figure(figsize=(10, 6))
plt.scatter(charged_off_loans['remaining_term'], charged_off_loans['remaining_loss'], color='red')
plt.title('Projected Loss Over Remaining Term of Charged Off Loans')
plt.xlabel('Remaining Term')
plt.ylabel('Projected Loss')
plt.grid(True)
plt.show()

# Calculating the total projected loss
total_projected_loss = charged_off_loans['remaining_loss'].sum()
print(f"Total Projected Loss of Charged Off Loans: ${total_projected_loss:.2f}")

In [ ]:

# Possible loss task -- given that results 0 ??error in above codes where I have imputed values from colums where there was high null count 

# Identifing the customers that are  currently behind on their loan payments
late_payments = data[data['loan_status'] == 'Late']

# Calculates the percentage of late payments compared to all loans
percentage_late_payments = (len(late_payments) / len(data)) * 100
print(f"Percentage of customers currently late on payments: {percentage_late_payments:.2f}%")

# Calculate total customers in the late payment bracket
total_late_customers = len(late_payments)

# Calculates the total loss if status changed to Charged Off
loss_if_charged_off = late_payments['recoveries'].sum() + late_payments['collection_recovery_fee'].sum()
print(f"Total loss if customers' status changed to Charged Off: ${loss_if_charged_off:.2f}")

# Calculates the projected loss if customers finish their loan terms
projected_loss_if_finished = late_payments['loan_amount'].sum() - late_payments['total_rec_prncp'].sum()
print(f"Projected loss if customers finish their loan terms: ${projected_loss_if_finished:.2f}")

# Calculates the percentage of total expected revenue from late payments and defaults
total_revenue = data['loan_amount'].sum()
late_payment_default_revenue = (loss_if_charged_off + projected_loss_if_finished) / total_revenue * 100
print(f"Percentage of total expected revenue from late payments and defaults: {late_payment_default_revenue:.2f}%")



In [ ]:
# Creating subsets for Charged Off and Late customers
charged_off = data[data['loan_status'] == 'Charged Off']
late_payments = data[data['loan_status'] == 'Late']

# Comparing 'grade' against Charged Off and Late customers
grade_comparison = pd.concat([charged_off['grade'].value_counts(normalize=True), late_payments['grade'].value_counts(normalize=True)], axis=1)
grade_comparison.columns = ['Charged Off', 'Late Payments']
print(grade_comparison)

# Comparing 'purpose' against Charged Off and Late customers
purpose_comparison = pd.concat([charged_off['purpose'].value_counts(normalize=True), late_payments['purpose'].value_counts(normalize=True)], axis=1)
purpose_comparison.columns = ['Charged Off', 'Late Payments']
print(purpose_comparison)

# Comparing 'home_ownership' against Charged Off and Late customers
home_comparison = pd.concat([charged_off['home_ownership'].value_counts(normalize=True), late_payments['home_ownership'].value_counts(normalize=True)], axis=1)
home_comparison.columns = ['Charged Off', 'Late Payments']
print(home_comparison)

# Visualising the comparisons I have made using bar charts


plt.figure(figsize=(10, 6))

plt.subplot(1, 3, 1)
grade_comparison.plot(kind='bar', ax=plt.gca())
plt.title('Grade Comparison')

plt.subplot(1, 3, 2)
purpose_comparison.plot(kind='bar', ax=plt.gca())
plt.title('Purpose Comparison')

plt.subplot(1, 3, 3)
home_comparison.plot(kind='bar', ax=plt.gca())
plt.title('Home Ownership Comparison')

plt.tight_layout()
plt.show()
